In [109]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import math
import time
import matplotlib.pyplot as plt
from datetime import datetime

In [111]:
#Initialize mediapipe pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.6)

In [112]:
# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))
    return round(angle, 2)

In [113]:
# Function to calculate Euclidean distance
def calc_distance(a, b):
    a, b = np.array(a), np.array(b)
    return np.linalg.norm(a - b)

In [114]:
# List to store biomechanical data per frame
report_data = []

In [115]:
#Preprocssing of the realtime webcam feed
def normalize_lighting(image):
    """Apply CLAHE to normalize lighting conditions."""
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l = clahe.apply(l)

    lab = cv2.merge((l, a, b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)


In [116]:
def apply_gamma_correction(image, gamma=1.2):
    """Improve clarity in low-light environments."""
    inv_gamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** inv_gamma * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(image, table)

In [117]:
def preprocess_frame(frame, size=(640, 480)):
       
   # 1. Edge-preserving denoise
    frame = cv2.bilateralFilter(frame, d=5, sigmaColor=50, sigmaSpace=50)

    # 2. Lighting normalization
    frame = normalize_lighting(frame)

    # 3. Gamma correction
    frame = apply_gamma_correction(frame, gamma=1.25)

    # 4. White balance (fixes yellow/blue tint)
    try:
        wb = cv2.xphoto.createSimpleWB()
        frame = wb.balanceWhite(frame)
    except:
        pass  # fallback if xphoto isn't available

    # 5. Resize after all corrections
    frame = cv2.resize(frame, size)

    return frame

In [118]:
def trunk_angle(shoulder, hip):
    """
    Returns how many degrees the torso tilts from vertical.
    0 deg = perfectly straight.
    Higher = more tilt.
    """
    sx, sy = shoulder[0], shoulder[1]
    hx, hy = hip[0], hip[1]

    dx = sx - hx
    dy = hy - sy   # y increases downward in image

    if dx == 0 and dy == 0:
        return None

    # angle between torso line and vertical
    angle = math.degrees(math.atan2(abs(dx), abs(dy)))
    return angle

In [119]:
# One Euro Filter for smoothing

class OneEuroFilter:
    def __init__(self, freq, min_cutoff=1.0, beta=0.007, d_cutoff=1.0):
        self.freq = freq
        self.min_cutoff = min_cutoff
        self.beta = beta
        self.d_cutoff = d_cutoff
        self.last_time = None
        self.x_prev = None
        self.dx_prev = None
    
    def alpha(self, cutoff):
        tau = 1.0 / (2 * math.pi * cutoff)
        te = 1.0 / self.freq
        return 1.0 / (1.0 + tau / te)

    def __call__(self, x):
        # Current timestamp
        t = time.time()
        if self.last_time is None:
            self.last_time = t
            self.x_prev = x
            self.dx_prev = 0.0
            return x
        
        # Estimate derivative
        dx = (x - self.x_prev) * self.freq
        a_d = self.alpha(self.d_cutoff)
        dx_hat = a_d * dx + (1 - a_d) * self.dx_prev
        
        # Update cutoff
        cutoff = self.min_cutoff + self.beta * abs(dx_hat)
        a = self.alpha(cutoff)
        
        # Filtered value
        x_hat = a * x + (1 - a) * self.x_prev
        
        # Save previous values
        self.x_prev = x_hat
        self.dx_prev = dx_hat
        self.last_time = t
        
        return x_hat


In [120]:
def compute_progression_metrics(df):
    metrics = {}

    if "body_angle" not in df.columns:
        raise ValueError("body_angle not found — posture progression cannot be computed")

    metrics["avg_posture_angle"] = df["body_angle"].mean()
    metrics["posture_std"] = df["body_angle"].std()

    if "posture_label" in df.columns:
        posture_dist = df["posture_label"].value_counts(normalize=True)
        metrics["good_posture_ratio"] = posture_dist.get("Good Posture", 0)
        metrics["lean_ratio"] = posture_dist.get("Lean Forward", 0)
        metrics["slouch_ratio"] = posture_dist.get("Slouched", 0)

    return metrics


In [121]:
def detect_progression_state(metrics):
    if metrics["good_posture_ratio"] > 0.7 and metrics["posture_std"] < 8:
        return "progressing"
    elif metrics["slouch_ratio"] > 0.4:
        return "regressing"
    else:
        return "plateau"


In [122]:
# Smoothing filters for metrics
freq = 60  # target FPS for smoothing
knee_filter = OneEuroFilter(freq)
hip_filter = OneEuroFilter(freq)
torso_leg_filter = OneEuroFilter(freq)
shoulder_width_filter = OneEuroFilter(freq)
hip_width_filter = OneEuroFilter(freq)
elbow_filter = OneEuroFilter(freq)
posture_angle_filter = OneEuroFilter(freq)
posture_score_filter = OneEuroFilter(freq)



In [123]:
# Start webcam capture
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.6, min_tracking_confidence=0.6) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
            
        # PREPROCESSING
        frame = preprocess_frame(frame)
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            # Get key points (Left side)
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]
            shoulder_z = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z
            
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].z]
            
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y,
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].z]
            
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
            hip_z = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z
            
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z]
            
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z]
            
            ear = [landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y,
                   landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].z]
            
            ear_z = landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].z

            # Get right side keypoints for width and ratio calculations
            shoulder_r = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,
                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z]
            
            hip_r = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z]
            
            # Calculate Angles
            elbow_angle = calculate_angle(shoulder, elbow, wrist)
            knee_angle = calculate_angle(hip, knee, ankle)
            hip_angle = calculate_angle(shoulder, hip, knee)

            #Posture classification
            body_angle = trunk_angle(shoulder, hip) 
            if body_angle is None:
               posture_label = "Unknown"
           # Thresholds (tuned to be accurate)
            elif body_angle < 10:
               posture_label = "Good Posture"
            elif body_angle < 20:
               posture_label = "Lean Forward"
            else:
               posture_label = "Slouched Backward"

           
            # Calculate widths
            shoulder_width = calc_distance(shoulder, shoulder_r)
            hip_width = calc_distance(hip, hip_r)
            
            # Calculate torso & leg ratio
            torso_center = np.mean([shoulder, shoulder_r], axis=0)
            hip_center = np.mean([hip, hip_r], axis=0)
            torso_length = calc_distance(torso_center, hip_center)
            leg_length = calc_distance(hip, ankle)
            torso_leg_ratio = round(torso_length / leg_length, 1) if leg_length != 0 else 0

            # Apply smoothing to all metrics
            knee_angle_s = knee_filter(knee_angle)
            elbow_angle_s = elbow_filter(elbow_angle)
            hip_angle_s = hip_filter(hip_angle)
            torso_leg_s = torso_leg_filter(torso_leg_ratio)
            shoulder_w_s = shoulder_width_filter(shoulder_width)
            hip_w_s = hip_width_filter(hip_width)
           
            # Display angles on frame (rendering)
            cv2.putText(image, f'Elbow Angle: {round(elbow_angle_s,1)}°', (50, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(image, f'Hip Angle: {round(hip_angle_s,1)}°', (50, 80), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(image, f'Knee Angle: { round(knee_angle_s,1)}°', (50, 110), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(image, f'Torso/Leg Ratio: {round(torso_leg_s,1)}', (50, 170), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(image, f'Shoulder Width: {round(shoulder_w_s,1)}', (50, 200), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(image, f'Hip Width: {round(hip_w_s,1)}', (50, 230), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA),
            cv2.putText(image, f"Posture Label: {posture_label}", (50, 255),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2, cv2.LINE_AA)

            
            # Draw pose landmarks
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=3),
                                      mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2))


            SESSION_ID = datetime.now().strftime("%Y%m%d_%H%M%S")

            
            # Log biomechanical data
            report_data.append({
                "session_id": SESSION_ID,
                "date": datetime.now().strftime("%Y-%m-%d"),
                'timestamp': datetime.now().strftime('%H:%M:%S'),
                'elbow_angle': round(elbow_angle_s,2),
                'hip_angle': round(hip_angle_s,2),
                'knee_angle': round(knee_angle_s,2),
                'shoulder_width': round(shoulder_w_s, 2),
                'hip_width': round(hip_w_s, 2),
                'torso_leg_ratio': round(torso_leg_s,2),
                'posture_label': posture_label,
                'body_angle': body_angle
            })
        
        # Show live video
        cv2.imshow('BioFitCoach - Real-Time Pose', image)

        # Exit condition
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [124]:
# Save report to CSV
if report_data:
    df = pd.DataFrame(report_data)
    df.to_csv("biomechanical_live_report.csv", index=False)
    print("Real-time biomechanical report saved as 'biomechanical_live_report.csv'")
    display(df.tail())
else:
    print("No pose data detected.")

Real-time biomechanical report saved as 'biomechanical_live_report.csv'


,session_id,date,timestamp,elbow_angle,hip_angle,knee_angle,shoulder_width,hip_width,torso_leg_ratio,posture_label,body_angle
1321,20260112_173605,2026-01-12,17:36:05,127.08,152.55,124.92,0.54,0.42,0.66,Good Posture,5.879543
1322,20260112_173605,2026-01-12,17:36:05,127.75,151.76,123.94,0.54,0.42,0.66,Good Posture,5.899711
1323,20260112_173605,2026-01-12,17:36:05,128.06,151.25,123.57,0.54,0.42,0.66,Good Posture,5.232124
1324,20260112_173605,2026-01-12,17:36:05,128.19,150.69,123.43,0.54,0.42,0.66,Good Posture,5.418423
1325,20260112_173605,2026-01-12,17:36:05,128.31,150.30,123.16,0.54,0.42,0.65,Good Posture,5.466098


In [125]:
df = pd.read_csv("biomechanical_live_report.csv")

if "session_id" in df.columns:
    recent_sessions = df.groupby("session_id").tail(50)
else:
    print(" session_id not found — falling back to last 200 frames")
    recent_sessions = df.tail(200)

In [126]:
# Feedback from the biomechanical analysis report data and exercises recommendation accordingly

In [127]:
from openai import OpenAI
client = OpenAI(api_key="enter key here") 



In [128]:
def biomech_findings(row):
    """
    Convert biomechanical metrics into clear, beginner-friendly biomechanical insights.
    """
    findings = []

    # 1. Torso–Leg Ratio (body leverage)
    if row['torso_leg_ratio'] < 0.65:
        findings.append("You have longer legs compared to your torso — this usually makes deep squats harder and benefits hinge movements like deadlifts.")
    elif row['torso_leg_ratio'] > 0.8:
        findings.append("You have a longer torso — this favors upright lifts and typically improves squat depth stability.")
    else:
        findings.append("Your torso and leg lengths are well balanced — most standard lifts should feel natural for your body.")

    # 2. Posture Findings
    if row['posture_label'] == "Good Posture":
        findings.append("Your posture appears neutral and well aligned — good spinal control and upper-back engagement.")
    elif row['posture_label'] == "Lean Forward":
        findings.append("You are leaning forward slightly — this may indicate tight chest muscles or weak upper-back stabilizers.")
    else:
        findings.append("You are slouching backward — likely due to weak core stability or limited hip extension control.")

    # 3. Shoulder/Hip Width Ratio (frame structure)
    ratio = row['shoulder_width'] / row['hip_width'] if row['hip_width'] != 0 else 1
    if ratio > 1.2:
        findings.append("Your shoulders are broader than your hips — pushing movements (pressing exercises) may feel strong and stable.")
    elif ratio < 0.9:
        findings.append("Your hips are wider relative to your shoulders — you may naturally excel in lower-body strength and stability movements.")
    else:
        findings.append("Your shoulder and hip widths are proportionate — you are well suited for a balanced full-body training approach.")

    return findings

In [129]:
def rule_based_recommendations(findings):
    """
    Maps biomechanical findings to practical exercise recommendations.
    Focuses on simple, effective, widely-used movements.
    """
    recs = []

    for f in findings:

        # Torso-Leg Leverage
        if "longer legs" in f:
            recs += ["Trap Bar Deadlift", "Sumo Deadlift", "Box Squat", "Romanian Deadlift"]

        elif "longer torso" in f:
            recs += ["Front Squat", "Goblet Squat", "Plank Variations", "Overhead Press"]

        elif "well balanced" in f:
            recs += ["Back Squat", "Conventional Deadlift", "Dumbbell Bench Press"]

        # Posture Issues
        elif "leaning forward" in f:
            recs += ["Seated Row", "Face Pull", "Chest Opener Stretch", "Rear Delt Fly"]

        elif "slouching backward" in f:
            recs += ["Dead Bug", "Bird Dog", "Glute Bridge", "Side Plank"]

        # Shoulder–Hip Structure
        elif "shoulders are broader" in f:
            recs += ["Incline Bench Press", "Dumbbell Shoulder Press", "Push-Ups"]

        elif "hips are wider" in f:
            recs += ["Hip Thrust", "Glute Bridge", "Bulgarian Split Squat", "Goblet Squat"]

    # Remove duplicates
    return list(set(recs))

In [131]:
def generate_llm_feedback(findings, recommendations,posture_label):
    """
    Generates a professional, easy-to-read fitness report
    using the OpenAI LLM.
    """

    # Convert Python lists to bullet points
    findings_text = "\n".join([f"- {f}" for f in findings])
    recs_text = ", ".join(recommendations)

    prompt = f"""
You are an elite biomechanics expert and professional strength training coach.
Create a short,accurate and motivating assessment report.

### INPUT:
Findings: {findings_text}
Recommended exercises list: {recs_text}
Posture label: {posture_label}

### INSTRUCTIONS:
1. Write in **simple language** for normal people (not athletes) and describe their biomechanical features.
2. Keep the tone positive, supportive, and easy to read.
3. Organize the output into 3 sections:
   - **Summary** (3–4 sentences max)
   - **Key Takeaways** (bullet points, 3–5 items)
   - **Recommended Exercises** (6–10 exercises, all well-known)

4. In the Summary:
   - Explain posture in plain English.
   - Mention if posture is: good, leaning forward, or slouched.
   - Give 1–2 lines of posture correction advice.

5. In Key Takeaways:
   - Mention body proportions (torso/leg ratio) simply.
   - Mention whether the user is naturally strong in pushing, pulling, squatting, etc.

6. In Recommended Exercises:
   - Only choose common, widely-known exercises.
   - Include 2–3 posture correction moves if posture is not good. 
   - Prefer exercises like:
     Squats, Deadlifts, Push-ups, Rows, Planks, Hip Thrusts, Lat Pulldown,
     Face Pulls, Reverse Fly, Chest Stretch, Cat-Cow, Wall Slides, etc.

7. Output MUST be valid JSON in the following structure:


Make sure the JSON is clean and not inside quotes.
"""

    response = client.chat.completions.create(
        model="gpt-5.2",      
        messages=[
            {"role": "system", "content": "You are a professional fitness coach and biomechanist."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.6
    )

    # Parse JSON safely
    raw = response.choices[0].message.content

    try:
        return json.loads(raw)
    except:
        return {"summary": raw, "key_takeaways": [], "recommended_exercises": []}

In [132]:
def generate_progression_plan(metrics, progression_state):
    prompt = f"""
You are a professional strength and conditioning coach.

User posture progression state: {progression_state}

Metrics:
- Average posture angle: {metrics['avg_posture_angle']:.1f}
- Posture stability (std): {metrics['posture_std']:.1f}
- Good posture ratio: {metrics['good_posture_ratio']:.2f}

TASK:
1. Explain what this progression state means in simple words
2. Suggest how to progress training over the next 2 weeks
3. Include posture-focused guidance
4. Keep it simple and encouraging
5. Avoid medical language
"""

    response = client.chat.completions.create(
        model="gpt-5.2",
        messages=[
            {"role": "system", "content": "You are a professional fitness coach."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.5
    )

    return response.choices[0].message.content


In [133]:
def generate_report_from_csv(csv_path):
 
    df = pd.read_csv(csv_path)

    # ---- 1. Select last 5 rows (or all if less than 5) ----
    if len(df) >= 5:
        recent = df.iloc[-5:]
    else:
        recent = df.copy()

    # ---- 2. Average numeric values ----
    avg_row = recent.mean(numeric_only=True)

    # ---- 3. Select most frequent posture label ----
    if "posture_label" in recent.columns:
        posture = recent["posture_label"].mode().iloc[0]
    else:
        posture = "Unknown"

    # Manually insert posture back into the averaged row
    avg_row["posture_label"] = posture

    # ---- 4. Generate findings & rule-based recs ----
    findings = biomech_findings(avg_row)
    recs = rule_based_recommendations(findings)

    # ---- 5. LLM feedback ----
    llm_output = generate_llm_feedback(findings, recs, posture)

    return {
        "averaged_metrics": avg_row.to_dict(),
        "findings": findings,
        "rule_based_recommendations": recs,
        "llm_report": llm_output
    }


In [134]:
#Results
report = generate_report_from_csv("biomechanical_live_report.csv")
print("Report Generated:\n", report)

metrics = compute_progression_metrics(recent_sessions)
state = detect_progression_state(metrics)
progression_plan = generate_progression_plan(metrics, state)

print("Progression State:", state)
print("Progression Plan:\n", progression_plan)

with open("progression_plan.txt", "w", encoding="utf-8") as f:
    f.write(progression_plan)



Report Generated:
 {'averaged_metrics': {'elbow_angle': 127.878, 'hip_angle': 151.31, 'knee_angle': 123.804, 'shoulder_width': 0.54, 'hip_width': 0.42000000000000004, 'torso_leg_ratio': 0.658, 'body_angle': 5.579179932960246, 'posture_label': 'Good Posture'}, 'findings': ['Your torso and leg lengths are well balanced — most standard lifts should feel natural for your body.', 'Your posture appears neutral and well aligned — good spinal control and upper-back engagement.', 'Your shoulders are broader than your hips — pushing movements (pressing exercises) may feel strong and stable.'], 'rule_based_recommendations': ['Conventional Deadlift', 'Dumbbell Shoulder Press', 'Dumbbell Bench Press', 'Push-Ups', 'Incline Bench Press', 'Back Squat'], 'llm_report': {'summary': '{\n  "Summary": "You have good posture: your spine looks neutral, with your chest and upper back naturally staying “tall” instead of slouched or leaning forward. Your torso and legs are well balanced, so most standard gym lif